## Plotting inference results for multidimensional posteriors 

In order to use this notebook you need data files stored in a folder `lfi-experiments/balancednetwork/data`. 

For saving the figures you should create a folder `lfi-experiments/balancednetwork/figures`

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pickle
import time
import scipy.stats as st
import os 
from lfimodels.balancednetwork.BalancedNetworkSimulator import BalancedNetwork
from lfimodels.balancednetwork.BalancedNetworkStats import BalancedNetworkStats
import delfi.distribution as dd

In [ ]:
from delfi.utils.viz import probs2contours, plot_pdf

## Load data and extract posterior object

data file should be in `lfi-experiments/balancednetwork/data`

In [ ]:
inference_method = 'basic'
filename = '1508026302673665_snpe_J_r4_ntrain300'

save_figure = False
path_to_save_folder = 'figures/'

time_str = filename[:filename.find('_')]
filename.find('_')
fullname = 'data/' + filename + '.p'

In [ ]:
# load data 
assert os.path.exists(fullname), 'path not found: {}. data file be in lfi-experiments/balancednetwork/data'.format(fullname)
with open(fullname, 'rb') as handle:
    result_dict = pickle.load(handle)

In [ ]:
# unpack values 
true_params, stats_obs, nrounds, ntrain, posterior, out, trn_data = result_dict.values()
dim_params = len(true_params[0])
assert dim_params > 1, 'this notebook is for inference on more than 1 parameter.'

In [ ]:
# extract the posterior 
n_components = len(posterior.a)
means = [posterior.xs[c].m for c in range(n_components)]
Ss = [posterior.xs[c].S for c in range(n_components)]

# mixing coefs 
mixing_coefs = posterior.a

# construct posterior
post = dd.mixture.MoG(posterior.a, means, Ss)

## Compare to true parameter 

We have generated the observed data ourselves so we do have the true parameter. The mean of the posterior should be close to it when evaluated for $x=x_{obs}$

In [ ]:
log_flag = False
resolution = .001
lims = [0.01, .1]
steps = int((lims[1] - lims[0])/ resolution)
theta = np.linspace(lims[0], lims[1], steps)
levels = [0.68, .95]

# get grid of sampling points
x, y = np.meshgrid(theta, theta)
# arrange samples in rows 
v = np.vstack((x.flatten(), y.flatten())).T

In [ ]:
plt.figure(figsize=(15, 10))
plot_idx = 1

for i in range(dim_params): 
    for j in range(dim_params): 
        
        if i == j: 
            z = post.eval(x=theta.T, log=log_flag, ii=[i])

            plt.subplot(dim_params, dim_params, plot_idx)
            plt.plot(theta, z)
            plt.axvline(x=true_params[0][i], color='C1', label=weight_labels[i])
        
        elif i < j:                     
            z = post.eval(x=v, log=log_flag, ii=[i, j]).reshape(x.shape)
            
            print('mass: ', z.sum() * resolution**2)
                
            contour_labels = probs2contours(z.flatten(), levels=levels).reshape(x.shape)
                        
            plt.subplot(dim_params, dim_params, plot_idx)
            plt.contourf(x, y, z)
            plt.contour(x, y, probs2contours(z, levels), levels, colors=('w', 'y'))
#            plt.colorbar() 
            plt.plot([true_params[0][i]], [true_params[0][j]], 'o', color='C1')
        plot_idx += 1

## Some figure without marginalization (deprecated)

In [ ]:
def get_delfi_grid_pdf(theta, delfi_obj, log=True, marglist=None): 
    """
    Get pdf of a whole grid of values 
    """
    x, y = np.meshgrid(theta, theta)
    z = np.zeros_like(x)
    for i in range(z.shape[0]): 
        # arrange the samples in rows 
        v = np.array([x[i, :], y[i, :]]).T
        # evaluate the pdf for rows of z
        z[i, :] = delfi_obj.eval(x=v, log=log, ii=marglist)
    return x, y, z

In [ ]:
plt.figure(figsize=(15, 10))
theta = np.linspace(0.01, 0.1, 100)
weight_labels = ['$J^{EE}$', '$J^{EI}$', '$J^{IE}$', '$J^{II}$']
plot_idx = 1
log_flag = False
for i in range(dim_params): 
    for j in range(dim_params): 
        if i==j: 
            
            # define a 1D MoG
            sub_means = [[means[c][i]] for c in range(n_components)]
            sub_cov = [[[Ss[c][i, j]]] for c in range(n_components)]
            pdf = dd.mixture.MoG(a=posterior.a, ms=sub_means, Ss=sub_cov)
            post_pdf = pdf.eval(theta[:, np.newaxis], log=log_flag)
            
            plt.subplot(dim_params, dim_params, plot_idx)            
            plt.plot(theta, post_pdf)
            plt.axvline(x=true_params[0][i], color='C1', label=weight_labels[i])
            plt.legend(prop=dict(size=15))
            
        elif i < j:            
            # define a 2D MoG
            sub_means = [[posterior.xs[c].m[i], posterior.xs[c].m[j]] for c in range(n_components)]
            sub_cov = [[[posterior.xs[c].S[i, i], posterior.xs[c].S[i, j]], 
                       [posterior.xs[c].S[j, i], posterior.xs[c].S[j, j]]] for c in range(n_components)]
            pdf = dd.mixture.MoG(a=posterior.a, ms=sub_means, Ss=sub_cov)
            x, y, z = get_delfi_grid_pdf(theta, delfi_obj=pdf, log=log_flag)
        
            plt.subplot(dim_params, dim_params, plot_idx)
            plt.contourf(x, y, z)
            plt.plot([true_params[0][i]], [true_params[0][j]], 'o', color='C1')
            plt.colorbar()
        plot_idx += 1 

if save_figure and os.path.exists(path_to_save_folder): 
    filename = time_str + '_{}_posteriormatrix_r{}_ntrain{}.pdf'.format(inference_method, nrounds, ntrain)
    plt.tight_layout()
    plt.savefig('figures/' + filename)

## Posterior predictive checking: 

Sample from the estimated posterior and simulate with the sampled parameters.

In [ ]:
m = BalancedNetwork(dim=dim_params, first_port=8010, verbose=False, n_servers=3)

In [ ]:
# generate a few samples and simulate 
n_samples = 5 
params = []
for i in range(n_samples): 
    params.append(post.gen())

In [ ]:
params = np.array([post.a[c] * means[c] for c in range(n_components)]).sum(axis=0)
print(params)

In [ ]:
# setup a simulator 
m = BalancedNetwork(n_servers=1, dim=4, duration=3., first_port=8010, parallel=False,
                   save_raster_plots=True, save_folder='/Users/Jan/Dropbox/Master/mackelab/code/lfi-experiments/balancednetwork/figures/simulation_raster_plots/')

In [ ]:
s = BalancedNetworkStats()

In [ ]:
try: 
    m.start_server()
except: 
    m.stop_server()
    print('server setup failed')

In [ ]:
# simulate 
data = m.gen([params])

In [ ]:
m.stop_server()

In [ ]:
# calculate summary stats
stats = []
for d in data: 
    stats.append(s.calc(d))

In [ ]:
stats = np.array(stats).squeeze()

In [ ]:
stats

In [ ]:
plt.plot(stats, 'o-', label='gen')
plt.plot(stats_obs.T, 'o-', label='obs')
plt.legend()

In [ ]:
plot_pdf(post, lims=np.repeat([0.01, 0.1], 4))

In [ ]:
post